In [30]:
import os
import random
import time

from reNN.base import *

from sklearn.ensemble import RandomForestClassifier

from tqdm import tqdm

In [46]:
ndims = 100
set_size = 10000

embedding_dir = "/Users/rob/ML_Data/x86_vectors/lstm/"+str(ndims)+"/"
#embedding_dir = "/Users/rob/ML_Data/x86_vectors/lstm/"+str(ndims)+"-raw/"
#embedding_dir = "/Users/rob/ML_Data/x86_vectors/byte_3grams/500/"
#label_dir = "/Users/rob/ML_Data/x86_vectors/byte_3grams/labels/"
reNN_dir = "/Users/rob/ML_Data/code_standard/"

In [47]:
all_reNN_files = []

for root, dir, files in os.walk(reNN_dir):
    for f in files:
        all_reNN_files.append(root+"/"+f)
        
print len(all_reNN_files)



6354


In [48]:
loader = reNN_binary()


collections = []
class1_funcs = []
class2_funcs = []

#class1_types = ["reactos", "windows10"]
#class2_types = ["Arch_Linux"]

class1_types = ["visual studio"]
class2_types = ["gcc"]

for fname in tqdm(all_reNN_files):
    try:
        loader = loader.load(fname)
    except:
        continue
    for func in loader.functions:
        collections.append(loader.cc_name)
        if loader.functions[func][1] > 20:
            if loader.cc_name in class1_types:
                class1_funcs.append(loader.md5+":"+loader.name+":"+func)
            elif loader.cc_name in class2_types:
                class2_funcs.append(loader.md5+":"+loader.name+":"+func)
                
print len(class1_funcs)
print len(class2_funcs)
print set(collections)
            

345092
747611
set([u'gcc', u'visual studio'])


In [49]:
def load_labels(label_file):
    labels = []
    with open(label_file, "r") as lfile:
        for line in lfile:
            labels.append(''.join([i + ":" for i in line.split(":")[1:]])[:-1].strip())
    return labels

In [50]:
def load_data_set(func_names, ndims):
    functions_by_hash = {}
    func_names.sort()
    counter = 0
    data_matrix = np.zeros((len(func_names), ndims))
    for f in func_names:
        file_hash = f.split(":")[0]
        function_name = ''.join([i + ":" for i in f.split(":")[2:]])[:-1]
        try:
            functions_by_hash[file_hash].append(function_name)
        except:
            functions_by_hash[file_hash] = [function_name]
    for file_hash in functions_by_hash:
        try:
            buff = np.load(embedding_dir+file_hash+".npy")
            buff_labels = load_labels(label_dir+file_hash+".labels")
        except:
            continue
        indexes = []
        for func_name in functions_by_hash[file_hash]:
            try:
                indexes.append(buff_labels.index(func_name))
            except:
                continue
        for i in indexes:
            data_matrix[counter] = buff[i]
            counter = counter + 1
    buff = data_matrix[:counter]
    return buff
        
        

In [51]:
def test_data_set
    scores = []
    for i in range(0,10):
        print "Starting loop %d..." % i
        start = time.time()
        class1_sample = random.sample(class1_funcs, 300000)
        class2_sample = random.sample(class2_funcs, 300000)

        class1_train_names = class1_sample[:240000]
        class2_train_names = class2_sample[:240000]
        class1_test_names = class1_sample[240000:]
        class2_test_names = class2_sample[240000:]

        class1_train = load_data_set(class1_train_names, ndims)
        class2_train = load_data_set(class2_train_names, ndims)
        train_classes = np.zeros(class1_train.shape[0]+class2_train.shape[0], dtype = np.int16)
        train_classes[class1_train.shape[0]:] = 1
        train_data = np.concatenate((class1_train, class2_train))
        print class1_train.shape
        print class2_train.shape
        print train_classes

        class1_test = load_data_set(class1_test_names, ndims)
        class2_test = load_data_set(class2_test_names, ndims)
        test_classes = np.zeros(class1_test.shape[0]+class2_test.shape[0], dtype = np.int16)
        test_classes[class1_test.shape[0]:] = 1
        test_data = np.concatenate((class1_test, class2_test))
        print class1_test.shape
        print class2_test.shape
        print test_classes

        classifier = RandomForestClassifier()
        classifier.fit(train_data, train_classes)

        scores.append(classifier.score(test_data, test_classes))
        print "Loop completed in %.3f seconds" % (time.time() - start)
    return scores

Starting loop 0...
(221865, 100)
(235590, 100)
[0 0 0 ..., 1 1 1]
(55522, 100)
(58922, 100)
[0 0 0 ..., 1 1 1]
Loop completed in 633.475 seconds
Starting loop 1...
(221932, 100)
(235635, 100)
[0 0 0 ..., 1 1 1]
(55418, 100)
(58950, 100)
[0 0 0 ..., 1 1 1]
Loop completed in 619.382 seconds
Starting loop 2...
(221954, 100)
(235646, 100)
[0 0 0 ..., 1 1 1]
(55422, 100)
(58919, 100)
[0 0 0 ..., 1 1 1]
Loop completed in 618.709 seconds
Starting loop 3...
(221819, 100)
(235710, 100)
[0 0 0 ..., 1 1 1]
(55528, 100)
(58879, 100)
[0 0 0 ..., 1 1 1]
Loop completed in 610.220 seconds
Starting loop 4...
(221904, 100)
(235600, 100)
[0 0 0 ..., 1 1 1]
(55465, 100)
(58941, 100)
[0 0 0 ..., 1 1 1]
Loop completed in 563.229 seconds
Starting loop 5...
(221798, 100)
(235720, 100)
[0 0 0 ..., 1 1 1]
(55546, 100)
(58888, 100)
[0 0 0 ..., 1 1 1]
Loop completed in 539.837 seconds
Starting loop 6...
(221960, 100)
(235601, 100)
[0 0 0 ..., 1 1 1]
(55444, 100)
(58948, 100)
[0 0 0 ..., 1 1 1]
Loop completed in 5

In [52]:
print "Accuracy: %.4f" % (sum(scores)/len(scores))

Accuracy: 0.9947


In [ ]:
print scores